[View in Colaboratory](https://colab.research.google.com/github/yingjie-wu/Reinforcement-Learning-StarAI/blob/master/Lesson_2_Exercise_3_Value_Iteration.ipynb)

In [0]:
          _____                _____                    _____                    _____                    _____                    _____          
         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
 /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
 \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/         

# Value Iteration Exercise

## Task

Implement value iteration algorithm:

Instead of doing multiple steps of Policy Evaluation to find the optimal state value only do a single step and update the policy immediately.

## Steps:

- For each state calculate its action values using current policy and value function.
- Identify the best action (i.e. the action with max value).
- Update policy to always select this particular action.
- Update state value function using selected action.
- Iterate until policy and state value functions converged to optimal.

### Hint:

Look one step ahead and evaluate your actions first.

## What we implement

$$
Q_*(s,a) = R(s,a) + \gamma \sum_{s \in S} P^a_{ss'} max(Q_*(s', a'))
$$

## Implementation

In [0]:
# !python -m pip install -e git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
# !python -m pip install gym
!pip install -e git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
!pip install gym

Obtaining rlenvs from git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
  Cloning https://github.com/star-ai/rl-environments.git to ./src/rlenvs
  Running setup.py develop for rlenvs
    100% |████████████████████████████████| 1.5MB 9.1MB/s 
    100% |████████████████████████████████| 1.0MB 13.1MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/ea/ec/dd/33bcc8801d345f0b640fced8a0864a7c8474828564bc5ccf70
Successfully built gym


In [0]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint

# Import below can all of a sudden break
# NOTE: if running locally, remove src.rlenvs from import
from src.rlenvs.rlenvs.envs.gridworld import GridworldEnv

In [0]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [0]:
def calculate_state_value(policy, state, env, V, discount_factor):
    """"
    Calculate state value assuming to take action with maximum expected value, 
    given possible actions, state, and current state value function.

    Args:
      policy: Policy - [S, A] matrix of probabilities of action A given state S
      env: Environment.
        env.P[s][a] return list of transition tuples (transition_probability, 
          next_state, reward, done).
      V: current state value function, V[s] return value for state s.
    """
    
    action_value = []
    # Look at the possible next actions
    for a, action_prob in enumerate(policy[state]):
        Q = 0
        # For each action, look at the possible next states...
        for prob, next_state, reward, done in env.P[state][a]:
            # Calculate the expected action value
            Q += prob * (reward + discount_factor * V[next_state])
        action_value.append(Q)
        
    # Take max expected action value
    v = max(action_value)    
        
    return v
  

def run_full_sweep(policy, env, V, discount_factor):
  """
  Run a full sweep over states.
  """
  new_V = np.zeros(env.nS)
  delta = 0
  # For each state, perform a "full backup"
  for s in range(env.nS):
      v = calculate_state_value(policy, s, env, V, discount_factor)
      # How much our value function changed (across any states)
      delta = max(delta, np.abs(v - V[s]))
      new_V[s] = v 
  return new_V, delta



def policy_update(policy, env, V):
    for state in range(env.nS):
      next_state_value = []
      for a, action_prob in enumerate(policy[state]):
          # For each action, look at the possible next states...
          #print("action:",a,"action_prob:",action_prob)
          for prob, next_state, reward, done in env.P[state][a]:
              # Calculate the expected value
              #print("prob:",prob,"next_state:",next_state,"reward:",reward,"done:",done)
              next_state_value.append(V[next_state])
      greedy_action = np.argmax(next_state_value)
      new_policy_for_state = [0,0,0,0]
      new_policy_for_state[greedy_action] = 1
      policy[state] = new_policy_for_state   
    return policy

  

def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS) 
    policy = np.zeros([env.nS, env.nA])
    
    ## iteration to find the optimal state value function
    i = 0
    while True:
      i+=1
      V, delta = run_full_sweep(policy, env, V, discount_factor)
      # Stop evaluating once our value function change is below a threshold
      if delta < theta or i>=10:
          print('stop at value iteration ', i)
          break 
    
    ## extract policy from the optimal state value function
    policy = policy_update(policy, env, V)

    return policy, V

In [0]:
policy, v = value_iteration(env)

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

stop at value iteration  4
Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [0]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [0]:
# Task 3, check policy
print("Policy Probability Distribution:")
print(policy)
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

